In [ ]:
import torch
from fourier_scaffold import (
    FourierScaffold,
    HammardSharpening,
    HammardShift,
    HammardShiftMatrix,
    HammardSharpening,
    ContractionSharpening,
    GaussianFourierSmoothing,
    GuassianFourierSmoothingMatrix,
)
from graph_utils import fourier_plot_probabilities_complex

In [ ]:
shapes = [[5, 5], [7, 7]]
device = "cuda"
scaffold = FourierScaffold(
    shapes=torch.tensor(shapes),
    D=1000**2,
    smoothing=GaussianFourierSmoothing(kernel_radii=[10] * 2, kernel_sigmas=[0.4] * 2),
    device=device,
)

In [ ]:
scaffold.velocity_shift(v=torch.tensor([1, 1]))

In [ ]:
scaffold.smooth()

In [ ]:
scaffold.sharpen()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.set_ylim(0,1)
fourier_plot_probabilities_complex(scaffold, ax)
plt.show()

In [ ]:
import itertools

for i, k in enumerate(
    itertools.product(
        *[list(range(scaffold.shapes[:, i].prod())) for i in range(scaffold.d)]
    )
):
    p = scaffold.get_probability(torch.tensor(k, device=scaffold.device))


In [ ]:
shapes = [[5, 5], [7, 7]]
device = "cuda"
scaffold = FourierScaffold(
    shapes=torch.tensor(shapes),
    D=1000,
    smoothing=GuassianFourierSmoothingMatrix(kernel_radii=[10] * 2, kernel_sigmas=[0.4] * 2),
    sharpening=ContractionSharpening(2),
    shift=HammardShiftMatrix(),
    device=device,
    representation='matrix'
)


In [ ]:
scaffold.sharpen()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.set_ylim(0,1)
fourier_plot_probabilities_complex(scaffold, ax)
plt.show()


In [ ]:
scaffold.velocity_shift(v=torch.tensor([1, 1]))

In [ ]:
import itertools

for i, k in enumerate(
    itertools.product(
        *[list(range(scaffold.shapes[:, i].prod())) for i in range(scaffold.d)]
    )
):
    p = scaffold.get_probability(torch.tensor(k, device=scaffold.device))
    print(p)